In [1]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
import hdbscan
from sklearn.decomposition import PCA
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch

In [2]:
print("GPU available:", torch.cuda.is_available())

GPU available: True


In [3]:
df = pd.read_csv("../data/stemmed_merged_kubu_02.csv") 
print(df.columns)

Index(['created_at', 'id_str', 'full_text', 'lang', 'location'], dtype='object')


In [4]:
df['full_text'] = df['full_text'].str.replace(r'(wk)+|\bsih\b|\bya\b', '', regex=True)

In [5]:
docs = df['full_text'].dropna().astype(str).tolist()

In [6]:
embedding_model = SentenceTransformer("distiluse-base-multilingual-cased-v1")
embedding_model.to('cuda' if torch.cuda.is_available() else 'cpu')

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Dense({'in_features': 768, 'out_features': 512, 'bias': True, 'activation_function': 'torch.nn.modules.activation.Tanh'})
)

In [7]:
pca_model = PCA(n_components=25)
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric='cosine')

hdbscan_model = hdbscan.HDBSCAN(
    min_cluster_size=15,
    metric='euclidean',
    cluster_selection_method='eom',
    prediction_data=True,
)


In [8]:
topic_model_hdbscan = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    calculate_probabilities=True,
    verbose=True
)

topics_hdbscan, probs = topic_model_hdbscan.fit_transform(docs)

topic_model_hdbscan.reduce_topics(docs, nr_topics=15)


2025-05-22 12:06:15,772 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/2951 [00:00<?, ?it/s]

2025-05-22 12:06:31,089 - BERTopic - Embedding - Completed ✓
2025-05-22 12:06:31,090 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-05-22 12:06:52,320 - BERTopic - Dimensionality - Completed ✓
2025-05-22 12:06:52,321 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-05-22 12:41:28,218 - BERTopic - Cluster - Completed ✓
2025-05-22 12:41:28,231 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-05-22 12:41:28,897 - BERTopic - Representation - Completed ✓
2025-05-22 12:41:30,239 - BERTopic - Topic reduction - Reducing number of topics
2025-05-22 12:41:30,277 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-05-22 12:41:30,612 - BERTopic - Representation - Completed ✓
2025-05-22 12:41:30,617 - BERTopic - Topic reduction - Reduced number of topics from 544 to 15


In [9]:
# Topic summary table
topic_info = topic_model_hdbscan.get_topic_info()
print(topic_info)

# Interactive visualizations
topic_model_hdbscan.visualize_topics()
topic_model_hdbscan.visualize_barchart(top_n_topics=15)
topic_model_hdbscan.visualize_heatmap()


    Topic  Count                                    Name  \
0      -1  32089              -1_prabowo_02_dukung_pilih   
1       0  27601                0_02_dukung_pilih_pasang   
2       1  16775      1_presiden_calon_indonesia_prabowo   
3       2  11446           2_prabowo_gibran_jokowi_orang   
4       3   4861             3_debat_data_calon_presiden   
5       4    533          4_israel_palestina_dukung_bela   
6       5    351           5_viral_dokter_porno_viralkan   
7       6    212  6_ekonomi_tumbuh_prabowo_infrastruktur   
8       7    126            7_online_message_direct_ojek   
9       8    125              8_edit_reformasi_cape_orde   
10      9     84               9_sistem_alat_utama_tahan   
11     10     70       10_stres_teori_prabowo_konspirasi   
12     11     68        11_energi_hijau_gibran_indonesia   
13     12     47              12_tulis_tuli_prabowo_buta   
14     13     22                    13_00am_jalan_tun_02   

                                       

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'x: %{x}<br>y: %{y}<br>Similarity Score: %{z}<extra></extra>',
              'name': '0',
              'type': 'heatmap',
              'x': [0_02_dukung_pilih, 1_presiden_calon_indonesia,
                    2_prabowo_gibran_jokowi, 3_debat_data_calon,
                    4_israel_palestina_dukung, 5_viral_dokter_porno,
                    6_ekonomi_tumbuh_prabowo, 7_online_message_direct,
                    8_edit_reformasi_cape, 9_sistem_alat_utama,
                    10_stres_teori_prabowo, 11_energi_hijau_gibran,
                    12_tulis_tuli_prabowo, 13_00am_jalan_tun],
              'xaxis': 'x',
              'y': [0_02_dukung_pilih, 1_presiden_calon_indonesia,
                    2_prabowo_gibran_jokowi, 3_debat_data_calon,
                    4_israel_palestina_dukung, 5_viral_dokter_porno,
                    6_ekonomi_tumbuh_prabowo, 7_online_message_direct,
                    8_edit_reformasi_cape, 9_sistem_alat_utama,
                    10_stres_teori_prabowo, 11_energi_hijau_gibran,
                    12_tulis_tuli_prabowo, 13_00am_jalan_tun],
              'yaxis': 'y',
              'z': {'bdata': ('+///////7z8sIhvyeYLnP/yhJ9b2CO' ... 'ugwrnJPzq4ROp3+NU/AgAAAAAA8D8='),
                    'dtype': 'f8',
                    'shape': '14, 14'}}],
    'layout': {'coloraxis': {'colorbar': {'title': {'text': 'Similarity Score'}},
                             'colorscale': [[0.0, 'rgb(247,252,240)'], [0.125,
                                            'rgb(224,243,219)'], [0.25,
                                            'rgb(204,235,197)'], [0.375,
                                            'rgb(168,221,181)'], [0.5,
                                            'rgb(123,204,196)'], [0.625,
                                            'rgb(78,179,211)'], [0.75,
                                            'rgb(43,140,190)'], [0.875,
                                            'rgb(8,104,172)'], [1.0,
                                            'rgb(8,64,129)']]},
               'height': 800,
               'hoverlabel': {'bgcolor': 'white', 'font': {'family': 'Rockwell', 'size': 16}},
               'legend': {'title': {'text': 'Trend'}},
               'margin': {'t': 60},
               'showlegend': True,
               'template': '...',
               'title': {'font': {'color': 'Black', 'size': 22},
                         'text': '<b>Similarity Matrix</b>',
                         'x': 0.55,
                         'xanchor': 'center',
                         'y': 0.95,
                         'yanchor': 'top'},
               'width': 800,
               'xaxis': {'anchor': 'y', 'constrain': 'domain', 'domain': [0.0, 1.0], 'scaleanchor': 'y'},
               'yaxis': {'anchor': 'x', 'autorange': 'reversed', 'constrain': 'domain', 'domain': [0.0, 1.0]}}
})

In [10]:
topic_info = topic_model_hdbscan.get_topic_info()
display(topic_info.head())        

,Topic,Count,Name,Representation,Representative_Docs
0,-1,32089,-1_prabowo_02_dukung_pilih,"[prabowo, 02, dukung, pilih, orang, calon, gib...",[prabowo dukung jokowi migrasi prabowo kasihan...
1,0,27601,0_02_dukung_pilih_pasang,"[02, dukung, pilih, pasang, menang, orang, kay...","[orang all in 02 aneh banget otak banget, 02 m..."
2,1,16775,1_presiden_calon_indonesia_prabowo,"[presiden, calon, indonesia, prabowo, gibran, ...",[pasang anies baswedan cak imin amin untung ki...
3,2,11446,2_prabowo_gibran_jokowi_orang,"[prabowo, gibran, jokowi, orang, menang, pinta...",[jokowi pilih prabowo prabowo cocok calon pimp...
4,3,4861,3_debat_data_calon_presiden,"[debat, data, calon, presiden, media, berita, ...",[menang debat anies ganjar menang pilih presid...


In [11]:
for tid in topic_info['Topic'][:15]:   # first 10 topics
    print(f"\nTopic {tid}")
    print(topic_model_hdbscan.get_topic(tid)[:15])  # top-15 words



Topic -1
[('prabowo', np.float64(0.048411360836698156)), ('02', np.float64(0.0443228913237586)), ('dukung', np.float64(0.032564888804654656)), ('pilih', np.float64(0.023733999301318916)), ('orang', np.float64(0.0210147958841153)), ('calon', np.float64(0.02097682603843247)), ('gibran', np.float64(0.019343423066841264)), ('pasang', np.float64(0.018882681493277154)), ('banget', np.float64(0.01631506840623672)), ('presiden', np.float64(0.014204891493625556))]

Topic 0
[('02', np.float64(0.07923414359691722)), ('dukung', np.float64(0.03462402711014225)), ('pilih', np.float64(0.026399803860566038)), ('pasang', np.float64(0.023593626184067493)), ('menang', np.float64(0.023311378250516088)), ('orang', np.float64(0.020773619919621098)), ('kayak', np.float64(0.01819755357382016)), ('banget', np.float64(0.01800404514235515)), ('buzzer', np.float64(0.01785336787451907)), ('calon', np.float64(0.0173564457302666))]

Topic 1
[('presiden', np.float64(0.0731264222896784)), ('calon', np.float64(0.04910

In [12]:
topic_model_hdbscan.visualize_barchart(top_n_topics=15)
topic_model_hdbscan.visualize_topics()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([[0, '02 | dukung | pilih | pasang | menang', 27601],
                                   [1, 'presiden | calon | indonesia | prabowo | gibran', 16775],
                                   [2, 'prabowo | gibran | jokowi | orang | menang', 11446],
                                   [3, 'debat | data | calon | presiden | media', 4861],
                                   [4, 'israel | palestina | dukung | bela | profesional', 533],
                                   [5, 'viral | dokter | porno | viralkan | 02', 351],
                                   [6, 'ekonomi | tumbuh | prabowo | infrastruktur | cipta', 212],
                                   [7, 'online | message | direct | ojek | judi', 126],
                                   [8, 'edit | reformasi | cape | orde | 02', 125],
                                   [9, 'sistem | alat | utama | tahan | beli', 84],
                                   [10, 'stres | teori | prabowo | konspirasi | 02', 70],
                                   [11, 'energi | hijau | gibran | indonesia | semangat', 68],
                                   [12, 'tulis | tuli | prabowo | buta | buku', 47],
                                   [13, '00am | jalan | tun | 02 | azlan', 22]], dtype=object),
              'hovertemplate': '<b>Topic %{customdata[0]}</b><br>%{customdata[1]}<br>Size: %{customdata[2]}',
              'legendgroup': '',
              'marker': {'color': '#B0BEC5',
                         'line': {'color': 'DarkSlateGrey', 'width': 2},
                         'size': {'bdata': '0WuHQbYs/RIVAl8B1AB+AH0AVABGAEQALwAWAA==', 'dtype': 'i2'},
                         'sizemode': 'area',
                         'sizeref': 17.250625,
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': {'bdata': 'OsgXQbibN0F0PXpBG+orQdnxP0G7Ew9BSjU+QVcWFUFhXANBgHN+QVOngEHI1XVBgniAQTh5BEE=',
                    'dtype': 'f4'},
              'xaxis': 'x',
              'y': {'bdata': 'T3MZQCE0+j80PixAIlYBQDFp0z80JiVAcnkUQPSGAkCvPYQ/AI+hP5/hJkChJBdAuyUMQJLC9j8=',
                    'dtype': 'f4'},
              'yaxis': 'y'}],
    'layout': {'annotations': [{'showarrow': False,
                                'text': 'D1',
                                'x': np.float32(6.978545),
                                'y': np.float32(1.9865769),
                                'yshift': 10},
                               {'showarrow': False,
                                'text': 'D2',
                                'x': np.float32(12.736252),
                                'xshift': 10,
                                'y': np.float32(3.0949912)}],
               'height': 650,
               'hoverlabel': {'bgcolor': 'white', 'font': {'family': 'Rockwell', 'size': 16}},
               'legend': {'itemsizing': 'constant', 'tracegroupgap': 0},
               'margin': {'t': 60},
               'shapes': [{'line': {'color': '#CFD8DC', 'width': 2},
                           'type': 'line',
                           'x0': np.float32(12.736252),
                           'x1': np.float32(12.736252),
                           'y0': np.float32(0.87816256),
                           'y1': np.float32(3.0949912)},
                          {'line': {'color': '#9E9E9E', 'width': 2},
                           'type': 'line',
                           'x0': np.float32(6.978545),
                           'x1': np.float32(18.493958),
                           'y0': np.float32(1.9865769),
                           'y1': np.float32(1.9865769)}],
               'sliders': [{'active': 0,
                            'pad': {'t': 50},
                            'steps': [{'args': [{'marker.color': [['red',
                                                                  '#B0BEC5',
                              

In [13]:
# df['created_at'] = pd.to_datetime(df['created_at'])

In [14]:
# topics_over_time = topic_model_hdbscan.topics_over_time(
#     docs=docs,
#     topics=topics_hdbscan,
#     timestamps=df['created_at'],
#     nr_bins=20,  # Adjust to control time granularity
#     evolution_tuning=True,
#     global_tuning=True
# )

In [15]:
# topic_model_hdbscan.visualize_topics_over_time(
#     topics_over_time, 
#     top_n_topics=10  # Number of top topics to show
# )


In [16]:
# Save results to CSV
df_topics = pd.DataFrame({"Document": docs, "Topic": topics_hdbscan})
df_topics.to_csv("topic_assignments_hdbscan.csv", index=False)

# Save model
# topic_model.save("bertopic_model_hdbscan")


In [17]:
from sklearn.cluster import KMeans

num_topics = 15
kmeans_model = KMeans(n_clusters=num_topics, random_state=42)

topic_model_kmeans = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=kmeans_model,
    calculate_probabilities=False,
    verbose=True
)

topics_kmeans, _ = topic_model_kmeans.fit_transform(docs)

topic_model_kmeans.visualize_barchart(top_n_topics=15)

2025-05-22 12:41:35,704 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/2951 [00:00<?, ?it/s]

2025-05-22 12:41:51,971 - BERTopic - Embedding - Completed ✓
2025-05-22 12:41:51,972 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-05-22 12:42:00,393 - BERTopic - Dimensionality - Completed ✓
2025-05-22 12:42:00,396 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-05-22 12:42:00,648 - BERTopic - Cluster - Completed ✓
2025-05-22 12:42:00,660 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-05-22 12:42:01,102 - BERTopic - Representation - Completed ✓


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.02157002575293172, 0.022511897939696122,
                    0.022999731988196617, 0.03637419656866257, 0.1537778624996075],
              'xaxis': 'x',
              'y': [jokowi  , orang  , dukung  , gibran  , prabowo  ],
              'yaxis': 'y'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.022527001943444126, 0.022783173581615072,
                    0.02349582176145096, 0.035369486444867086, 0.08621333534597744],
              'xaxis': 'x2',
              'y': [jokowi  , orang  , menang  , gibran  , prabowo  ],
              'yaxis': 'y2'},
             {'marker': {'color': '#CC79A7'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.0236106253024134, 0.02639760647336985, 0.03854952866626825,
                    0.04270501963526907, 0.06086855409374326],
              'xaxis': 'x3',
              'y': [pilih  , pasang  , dukung  , orang  , 02  ],
              'yaxis': 'y3'},
             {'marker': {'color': '#E69F00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.02608581430326582, 0.02939976801195413, 0.03669365230017733,
                    0.05164785732427692, 0.09832469893794103],
              'xaxis': 'x4',
              'y': [calon  , pilih  , pasang  , dukung  , 02  ],
              'yaxis': 'y4'},
             {'marker': {'color': '#56B4E9'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.018675880920272964, 0.03681139975224698,
                    0.03686097941223856, 0.060310030919203866, 0.17103010684744294],
              'xaxis': 'x5',
              'y': [banget  , buzzer  , pilih  , dukung  , 02  ],
              'yaxis': 'y5'},
             {'marker': {'color': '#009E73'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.0336720330941432, 0.03605039919159745, 0.04096755697022014,
                    0.04631079876923086, 0.06668982706535977],
              'xaxis': 'x6',
              'y': [kerja  , indonesia  , agama  , kampanye  , 02  ],
              'yaxis': 'y6'},
             {'marker': {'color': '#F0E442'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.03272417877670968, 0.04205940571338859,
                    0.044160604099783236, 0.06443767299101417, 0.06592451537303817],
              'xaxis': 'x7',
              'y': [video  , survei  , suara  , 02  , kayak  ],
              'yaxis': 'y7'},
             {'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.060052657231724586, 0.06294257834685688,
                    0.06874355141964165, 0.06952840268637189, 0.07072482459580468],
              'xaxis': 'x8',
              'y': [indonesia  , prabowo  , raka  , rakabuming  , gibran  ],
              'yaxis': 'y8'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.038960334018858754, 0.052678717566177286,
                    0.06050589241932973, 0.10385702562868596, 0.1479025378710047],
              'xaxis': 'x9',
              'y': [pilih  , 02  , wakil  , calon  , presiden  ],
              'yaxis': 'y9'},
             {'marker': {'color': '#CC79A7'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.03784350628317558, 0.04179602738532244, 0.04847688987537243,
                    0.05261004344496173, 0.1354409228225501],
              'xaxis': 'x10',
              'y': [gratis  , calon  , program  , 02  , debat  ],
              'yaxis': 'y10'},
             {'marker': {'color': '#E69F00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.04791725588904585, 0.061631363790

In [18]:
# Topic summary table
topic_info = topic_model_kmeans.get_topic_info()
print(topic_info)

# Interactive visualizations
topic_model_kmeans.visualize_topics()
topic_model_kmeans.visualize_barchart(top_n_topics=15)
topic_model_kmeans.visualize_heatmap()


    Topic  Count                                 Name  \
0       0  13797        0_prabowo_gibran_dukung_orang   
1       1  10325        1_prabowo_gibran_menang_orang   
2       2  10032             2_02_orang_dukung_pasang   
3       3   9214             3_02_dukung_pasang_pilih   
4       4   9089             4_02_dukung_pilih_buzzer   
5       5   6783        5_02_kampanye_agama_indonesia   
6       6   6692              6_kayak_02_suara_survei   
7       7   6178     7_gibran_rakabuming_raka_prabowo   
8       8   4486            8_presiden_calon_wakil_02   
9       9   4091             9_debat_02_program_calon   
10     10   3959      10_presiden_prabowo_calon_pilih   
11     11   3697             11_menang_02_teman_panik   
12     12   2157   12_politik_israel_palestina_polisi   
13     13   2055  13_demokrasi_partai_juang_indonesia   
14     14   1855      14_debat_prabowo_presiden_calon   

                                       Representation  \
0   [prabowo, gibran, dukung, 

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'coloraxis': 'coloraxis',
              'hovertemplate': 'x: %{x}<br>y: %{y}<br>Similarity Score: %{z}<extra></extra>',
              'name': '0',
              'type': 'heatmap',
              'x': [0_prabowo_gibran_dukung, 1_prabowo_gibran_menang,
                    2_02_orang_dukung, 3_02_dukung_pasang, 4_02_dukung_pilih,
                    5_02_kampanye_agama, 6_kayak_02_suara,
                    7_gibran_rakabuming_raka, 8_presiden_calon_wakil,
                    9_debat_02_program, 10_presiden_prabowo_calon,
                    11_menang_02_teman, 12_politik_israel_palestina,
                    13_demokrasi_partai_juang, 14_debat_prabowo_presiden],
              'xaxis': 'x',
              'y': [0_prabowo_gibran_dukung, 1_prabowo_gibran_menang,
                    2_02_orang_dukung, 3_02_dukung_pasang, 4_02_dukung_pilih,
                    5_02_kampanye_agama, 6_kayak_02_suara,
                    7_gibran_rakabuming_raka, 8_presiden_calon_wakil,
                    9_debat_02_program, 10_presiden_prabowo_calon,
                    11_menang_02_teman, 12_politik_israel_palestina,
                    13_demokrasi_partai_juang, 14_debat_prabowo_presiden],
              'yaxis': 'y',
              'z': {'bdata': ('AQCAP2iSYD/0axg/VkkePwqCAD8YOQ' ... '5JP8JDNz8mTvA+ns0WP2c6HD8BAIA/'),
                    'dtype': 'f4',
                    'shape': '15, 15'}}],
    'layout': {'coloraxis': {'colorbar': {'title': {'text': 'Similarity Score'}},
                             'colorscale': [[0.0, 'rgb(247,252,240)'], [0.125,
                                            'rgb(224,243,219)'], [0.25,
                                            'rgb(204,235,197)'], [0.375,
                                            'rgb(168,221,181)'], [0.5,
                                            'rgb(123,204,196)'], [0.625,
                                            'rgb(78,179,211)'], [0.75,
                                            'rgb(43,140,190)'], [0.875,
                                            'rgb(8,104,172)'], [1.0,
                                            'rgb(8,64,129)']]},
               'height': 800,
               'hoverlabel': {'bgcolor': 'white', 'font': {'family': 'Rockwell', 'size': 16}},
               'legend': {'title': {'text': 'Trend'}},
               'margin': {'t': 60},
               'showlegend': True,
               'template': '...',
               'title': {'font': {'color': 'Black', 'size': 22},
                         'text': '<b>Similarity Matrix</b>',
                         'x': 0.55,
                         'xanchor': 'center',
                         'y': 0.95,
                         'yanchor': 'top'},
               'width': 800,
               'xaxis': {'anchor': 'y', 'constrain': 'domain', 'domain': [0.0, 1.0], 'scaleanchor': 'y'},
               'yaxis': {'anchor': 'x', 'autorange': 'reversed', 'constrain': 'domain', 'domain': [0.0, 1.0]}}
})

In [19]:
topic_info = topic_model_kmeans.get_topic_info()
display(topic_info.head())        

,Topic,Count,Name,Representation,Representative_Docs
0,0,13797,0_prabowo_gibran_dukung_orang,"[prabowo, gibran, dukung, orang, jokowi, pilih...","[pikir dukung prabowo gibran kasihan coba, duk..."
1,1,10325,1_prabowo_gibran_menang_orang,"[prabowo, gibran, menang, orang, jokowi, anies...",[dukung ganjar amin alih prabowo gibran menang...
2,2,10032,2_02_orang_dukung_pasang,"[02, orang, dukung, pasang, pilih, banget, cal...",[prabowo orang pasang calon 02 mampu personal ...
3,3,9214,3_02_dukung_pasang_pilih,"[02, dukung, pasang, pilih, calon, buzzer, ban...","[prabowo salah duit dukung 02, dukung dukung p..."
4,4,9089,4_02_dukung_pilih_buzzer,"[02, dukung, pilih, buzzer, banget, bang, kala...","[dukung 02 nyebokin banget, 02 pilih takut, sa..."


In [20]:
for tid in topic_info['Topic'][:15]:   # first 10 topics
    print(f"\nTopic {tid}")
    print(topic_model_kmeans.get_topic(tid)[:15])  # top-15 words



Topic 0
[('prabowo', np.float64(0.1537778624996075)), ('gibran', np.float64(0.03637419656866257)), ('dukung', np.float64(0.022999731988196617)), ('orang', np.float64(0.022511897939696122)), ('jokowi', np.float64(0.02157002575293172)), ('pilih', np.float64(0.019958625829038668)), ('ganjar', np.float64(0.01908576024333069)), ('anies', np.float64(0.01744948740431866)), ('mas', np.float64(0.015853876317828444)), ('pimpin', np.float64(0.015451577629413748))]

Topic 1
[('prabowo', np.float64(0.08621333534597744)), ('gibran', np.float64(0.035369486444867086)), ('menang', np.float64(0.02349582176145096)), ('orang', np.float64(0.022783173581615072)), ('jokowi', np.float64(0.022527001943444126)), ('anies', np.float64(0.02073026287562034)), ('ganjar', np.float64(0.020071821097078755)), ('dukung', np.float64(0.018567712473022528)), ('pilih', np.float64(0.018179818587281547)), ('rakyat', np.float64(0.01777177858283392))]

Topic 2
[('02', np.float64(0.06086855409374326)), ('orang', np.float64(0.042

In [21]:
topic_model_kmeans.visualize_barchart(top_n_topics=15)
topic_model_kmeans.visualize_topics()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([[0, 'prabowo | gibran | dukung | orang | jokowi', 13797],
                                   [1, 'prabowo | gibran | menang | orang | jokowi', 10325],
                                   [2, '02 | orang | dukung | pasang | pilih', 10032],
                                   [3, '02 | dukung | pasang | pilih | calon', 9214],
                                   [4, '02 | dukung | pilih | buzzer | banget', 9089],
                                   [5, '02 | kampanye | agama | indonesia | kerja', 6783],
                                   [6, 'kayak | 02 | suara | survei | video', 6692],
                                   [7, 'gibran | rakabuming | raka | prabowo | indonesia', 6178],
                                   [8, 'presiden | calon | wakil | 02 | pilih', 4486],
                                   [9, 'debat | 02 | program | calon | gratis', 4091],
                                   [10, 'presiden | prabowo | calon | pilih | wakil', 3959],
                                   [11, 'menang | 02 | teman | panik | gara', 3697],
                                   [12, 'politik | israel | palestina | polisi | 02', 2157],
                                   [13, 'demokrasi | partai | juang | indonesia | demokrat', 2055],
                                   [14, 'debat | prabowo | presiden | calon | gibran', 1855]], dtype=object),
              'hovertemplate': '<b>Topic %{customdata[0]}</b><br>%{customdata[1]}<br>Size: %{customdata[2]}',
              'legendgroup': '',
              'marker': {'color': '#B0BEC5',
                         'line': {'color': 'DarkSlateGrey', 'width': 2},
                         'size': {'bdata': '5TVVKDAn/iOBI38aJBoiGIYR+w93D3EObQgHCD8H', 'dtype': 'i2'},
                         'sizemode': 'area',
                         'sizeref': 8.623125,
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': {'bdata': '3Z4NQTLBDUGbDktAJx0fQJdYKECppS1Ai52AQOcyCEEojhJAXLqOQLC4AUHlIApAas41QBKVM0DNyJZA',
                    'dtype': 'f4'},
              'xaxis': 'x',
              'y': {'bdata': 'TznGQMlnukDTlCTBOU00wc/AMcHBHRnB6iggwUxpr0DxrxfBWxIcwdrrs0ACGzjBgR0QwUPJCMFB2xfB',
                    'dtype': 'f4'},
              'yaxis': 'y'}],
    'layout': {'annotations': [{'showarrow': False,
                                'text': 'D1',
                                'x': np.float32(1.834519),
                                'y': np.float32(-3.0544565),
                                'yshift': 10},
                               {'showarrow': False,
                                'text': 'D2',
                                'x': np.float32(6.011568),
                                'xshift': 10,
                                'y': np.float32(7.12367)}],
               'height': 650,
               'hoverlabel': {'bgcolor': 'white', 'font': {'family': 'Rockwell', 'size': 16}},
               'legend': {'itemsizing': 'constant', 'tracegroupgap': 0},
               'margin': {'t': 60},
               'shapes': [{'line': {'color': '#CFD8DC', 'width': 2},
                           'type': 'line',
                           'x0': np.float32(6.011568),
                           'x1': np.float32(6.011568),
                           'y0': np.float32(-13.232583),
                           'y1': np.float32(7.12367)},
                          {'line': {'color': '#9E9E9E', 'width': 2},
                           'type': 'line',
                           'x0': np.float32(1.834519),
                           'x1': np.float32(10.188617),
                           'y0': np.float32(-3.0544565),
                           'y1': np.float32(-3.0544565)}],
               'sliders': [{'active': 0,
                            'pad': {'t': 50},
                            'steps': [{'args': [{'marker.color': [['red',
  

In [22]:
# Save results to CSV
df_topics = pd.DataFrame({"Document": docs, "Topic": topics_kmeans})
df_topics.to_csv("topic_assignments_kmeans.csv", index=False)

# Save model
# topic_model.save("bertopic_model_kmeans")

In [23]:
topics_over_time_kmeans = topic_model_kmeans.topics_over_time(
    docs=docs,
    topics=topics_hdbscan,
    timestamps=df['created_at'],
    nr_bins=20,
    evolution_tuning=True,
    global_tuning=True
)

topic_model_hdbscan.visualize_topics_over_time(topics_over_time_kmeans, top_n_topics=10)

0it [00:00, ?it/s]


IndexError: index (540) out of range

In [ ]:
print("HDBSCAN Topics:", len(set(topics_hdbscan)) - (1 if -1 in topics_hdbscan else 0))
print("KMeans Topics:", len(set(topics_kmeans)))

# Outlier analysis
print("HDBSCAN Outliers:", topics_hdbscan.count(-1))  # KMeans won't have outliers

HDBSCAN Topics: 205
KMeans Topics: 15
HDBSCAN Outliers: 9996


In [ ]:
import pandas as pd
from collections import Counter

# Count documents per topic
hdbscan_counts = Counter(topics_hdbscan)
kmeans_counts = Counter(topics_kmeans)

# Create DataFrame to compare
comparison_df = pd.DataFrame({
    "HDBSCAN": pd.Series(hdbscan_counts),
    "KMeans": pd.Series(kmeans_counts)
}).fillna(0).astype(int)

comparison_df.sort_index()


,HDBSCAN,KMeans
-1,9996,0
0,941,3525
1,903,3463
2,792,3304
3,789,2851
...,...,...
200,16,0
201,16,0
202,15,0
203,15,0
